<a href="https://colab.research.google.com/github/pksvv/lang-transformers/blob/main/TF_SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Data Preprocessing
* TF input pipeline
* Building and Training
* Making Predictions

# Dataset

https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/data

In [1]:
pip install kaggle

In [2]:
import kaggle

OSError: ignored

In [12]:
! cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [14]:
ls -alrth ~

total 72K
-r-xr-xr-x 1 root root  254 Jan  1  2000 .tmux.conf*
-r-xr-xr-x 1 root root 1.2K Jan  1  2000 .bashrc*
-rw-r--r-- 1 root root  148 Aug 17  2015 .profile
drwxr-xr-x 3 root root 4.0K Dec  3 14:33 .gsutil/
drwxr-xr-x 5 root root 4.0K Dec 16 14:10 .ipython/
drwx------ 1 root root 4.0K Dec 16 14:10 .local/
drwx------ 2 root root 4.0K Dec 16 14:10 .jupyter/
drwxr-xr-x 1 root root 4.0K Dec 16 14:10 .config/
drwxr-xr-x 4 root root 4.0K Dec 16 14:11 .npm/
drwxr-xr-x 1 root root 4.0K Dec 16 14:13 .cache/
drwxr-xr-x 2 root root 4.0K Dec 31 04:42 .keras/
drwxr-xr-x 2 root root 4.0K Dec 31 04:47 .ipynb_checkpoints/
drwxr-xr-x 3 root root 4.0K Dec 31 04:47 .kaggle/
-rw-r--r-- 1 root root   67 Dec 31 04:50 kaggle.json
drwx------ 1 root root 4.0K Dec 31 04:51 ./
drwxr-xr-x 1 root root 4.0K Dec 31 04:51 ../


In [15]:
mv ~/kaggle.json ~/.kaggle/kaggle.json

In [20]:
ls ~/.kaggle

kaggle/  kaggle.json
